In [8]:
import torch
import transformers
from transformers import AutoTokenizer, BertForSequenceClassification
import collections
from tqdm import tqdm
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

cuda = torch.device('cuda') 

In [17]:
-(1<<10)

-1024

In [9]:
import json
import gzip
import os
os.getcwd()
import pandas as pd
import datetime
import re 
import random 
import numpy as np

In [10]:
def deal_long_document(inputs, index, net_1, tfidf_weight):
    doc_vec = torch.zeros(768)
    for x in range(inputs["input_ids"].shape[0] // 512 + 1):
        input_sub = {}
        try:
            input_sub["input_ids"] = inputs["input_ids"][:,x*512:(x+1)*512].cuda()
            input_sub["token_type_ids"] = inputs["token_type_ids"][:,x*512:(x+1)*512].cuda() 
            input_sub["attention_mask"] = inputs["attention_mask"][:,x*512:(x+1)*512].cuda()
            outputs = net_1(**input_sub).hidden_states[0][0].cpu()
            torch.cuda.empty_cache()
            vec = torch.Tensor(tfidf_weight[index][x*512:(x+1)*512]).matmul(outputs).data
        except:
            input_sub["input_ids"] = inputs["input_ids"][:,x*512:].cuda()
            input_sub["token_type_ids"] = inputs["token_type_ids"][:,x*512:].cuda() 
            input_sub["attention_mask"] = inputs["attention_mask"][:,x*512:].cuda()  
            outputs = net_1(**input_sub).hidden_states[0][0].cpu()
            torch.cuda.empty_cache()
            vec = torch.Tensor(tfidf_weight[index][x*512:]).matmul(outputs).data
        doc_vec += vec
    return doc_vec

def CalculateTFidf(data):

    data_str = list(map(lambda x:x[0],data))
    #data_str = data
    for i in range(len(data_str)):
        for j in range(len(data_str[i])):
            data_str[i][j] = str(data_str[i][j])
    dictionary = Dictionary(data_str)
    corpus = list(map(dictionary.doc2bow,data_str))
    model = TfidfModel(corpus)
    corpus_tfidf = model[corpus]
    tfidf_weight = []
    for i in tqdm(range(len(data_str))):
        tfidf_vec = []
        dict_id_2_tfidf = dict(zip([x[0] for x in corpus_tfidf[i]], [x[1] for x in corpus_tfidf[i]]))
        for token in dictionary.doc2idx(data_str[i]):
            try:
                tfidf_vec.append(dict_id_2_tfidf[token])
            except:
                tfidf_vec.append(0.)
        tfidf_weight.append(tfidf_vec)
    return tfidf_weight

In [11]:
def Calculate_item_embedding(df):
    Ids = list(df.asin.values)
    Descriptions = list(df.description.values)
    documents = [tokenizer(x, return_tensors="pt")["input_ids"].tolist() for x in Descriptions]
    tfidf_weight = CalculateTFidf(documents)
    documents = np.array(documents)

    vec_list = []
    for i in tqdm(range(len(documents))):
        inputs = tokenizer(Descriptions[i], return_tensors="pt")
        if inputs["input_ids"].shape[1] <= 512: 
            inputs["input_ids"] = inputs["input_ids"].cuda()
            inputs["token_type_ids"] = inputs["token_type_ids"].cuda()
            inputs["attention_mask"] = inputs["attention_mask"].cuda()
            outputs = net_1(**inputs).hidden_states[0][0].cpu()
            torch.cuda.empty_cache()
            doc_vec = torch.Tensor(tfidf_weight[i]).matmul(outputs).data
        else:
            doc_vec = deal_long_document(inputs, i, net_1, tfidf_weight)
        vec_list.append(list(doc_vec.data / inputs["input_ids"].shape[1]))

    vec_list_2 = []
    for vector in vec_list:
        vec_list_2.append([float(value) for value in vector])

    dict_Id2vec = dict(zip(Ids, vec_list_2))

    return dict_Id2vec


In [5]:
# Download the pretrained BERT model.
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
net_1 = transformers.BertForPreTraining.from_pretrained('bert-base-uncased', output_hidden_states=True).cuda()
net_1.eval()

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [6]:
work_path = "./"

In [7]:
# The following .csv files can be download from https://cseweb.ucsd.edu/~jmcauley/datasets/amazon/links.html
side_files = ['sub_side_Books.csv', 'sub_side_Sports_and_Outdoors.csv', 'sub_side_Video_Games.csv']

In [15]:
for i in range(len(side_files)):
    df = pd.read_csv(work_path + side_files[i])
    Dict_Id2Vec = Calculate_item_embedding(df)
    np.save(f"./Dict_item2vec_{side_files[i][9:-4]}.npy",Dict_Id2Vec) 

100%|██████████| 20720/20720 [00:03<00:00, 5210.36it/s]
C:\Users\psy62\AppData\Local\Temp\ipykernel_21688\2782099054.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  documents = np.array(documents)
100%|██████████| 20720/20720 [09:24<00:00, 36.71it/s] 
Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 20708/20708 [00:03<00:00, 6574.98it/s]
C:\Users\psy62\AppData\Local\Temp\ipykernel_21688\2782099054.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must s

# Re-sample sub datasets

In [13]:
Vec_lst = [
 'Dict_item2vec_Books.npy',
 'Dict_item2vec_Movies_and_TV.npy',
 'Dict_item2vec_Sports_and_Outdoors.npy',
 'Dict_item2vec_Video_Games.npy']

# The following .csv files can be download from https://cseweb.ucsd.edu/~jmcauley/datasets/amazon/links.html
Rating_lst = [
 '5star_Books.csv',
 '5star_Movies_and_TV.csv',
 '5star_Sports_and_Outdoors.csv',
 '5star_Video_Games.csv'
]

def Data_Pareprocess(df_rate, start_time):
    
    df_rate = df_rate[["account_id", "deal_id", "UnixTime"]]
    df_rate["time"] = pd.to_datetime(df_rate["UnixTime"], unit="s")
    df_rate["time"] = df_rate.time.dt.date

    min_time = df_rate.time.min()
    max_time = df_rate.time.max()
    print(f"Time Zone: ({min_time}～{max_time})")
    
    df_T = df_rate
    
    df_rate_sub = df_T.loc[df_T.time > start_time]
    
    return df_rate_sub

def FilterDataframeByList(target_data, col, col_name):
    df_col = pd.DataFrame(col)
    df_col.columns = [col_name]
    output = pd.merge(target_data, df_col, how='inner', left_on=[col_name], right_on=[col_name])
    return output



def PrintInfo(dataset):
    n_users = len(dataset.account_id.unique())
    n_items = len(dataset.deal_id.unique())
    n_interaction = dataset.shape[0]
    print(f'user number: {n_users}')
    print(f'item number: {n_items}')
    print(f'interaction number: {n_interaction}')

Lst_dict_item2vec = []
for i in range(len(Vec_lst)):
    Lst_dict_item2vec.append(np.load("./" + Vec_lst[i], allow_pickle=True).item())


In [71]:
# Dense scenario

for i in range(4):
    df = pd.read_csv("./Dataset/" + Rating_lst[i])
    item_lst_r = df.deal_id.unique()
    item_lst = list(Lst_dict_item2vec[i].keys())
    item_lst = set(item_lst) & set(item_lst_r)
    df = FilterDataframeByList(df, item_lst, "deal_id")
    
    filted_user_1 = df.account_id.value_counts().keys()[:(df.account_id.value_counts().values >= 5).sum()]
    df = FilterDataframeByList(df, filted_user_1, "account_id")

    filted_user_1 = df.account_id.value_counts().keys()[len(df.deal_id.unique()) - (df.account_id.value_counts().values <= 15).sum():]
    df = FilterDataframeByList(df, filted_user_1[:8000], "account_id")
    df.to_csv(f"./Dataset/Dataset_s_{Rating_lst[i][6:-4]}_8000u.csv")
    PrintInfo(df)

user number: 8000
item number: 11900
interaction number: 63188
user number: 8000
item number: 8490
interaction number: 55735
user number: 8000
item number: 11749
interaction number: 43999
user number: 6650
item number: 6659
interaction number: 45157
